In [1]:
import dgipy

## Get Drug-Gene Interaction (DGI) Data
Use DGIpy to query DGIdb for DGI data and drug applications for drugs of interest. The appNo's will be used to pull relevant FDA labels from the JSON dataset. The DGI data will eventually be linked up with whatever is extracted from the JSON dataset. The drugs of interest for this experiment are:  
  
- **Imatinib** (favorite test case)
- **Encorafenib** (complex targeted therapy)
- **Cisplatin** (common cancer therapy)
- **Aspirin** (common therapy)
- **FAKE** (fake)

In [2]:
drugs = ['imatinib','encorafenib','cisplatin','aspirin','fake']
dgi_data = dgipy.get_interactions(drugs, search='drugs')
dgi_data

,drug,gene,approval,score,interaction_attributes,source,pmid
0,IMATINIB,FIP1L1,True,2.199452,Fusion Protein: FIP1L1:PDGFRA,FDA Pharmacogenomic Biomarkers | PharmGKB - Th...,24433361
1,IMATINIB,ABL1,True,0.119476,Mechanism of Action: Inhibition| Endogenous Dr...,The ChEMBL Bioactivity Database | Drug Target ...,
2,IMATINIB,BIRC5,True,0.016854,,NCI Cancer Gene Index,16254145
3,IMATINIB,ULK3,True,0.733151,,PharmGKB - The Pharmacogenomics Knowledgebase,15073101
4,IMATINIB,NF1,True,0.031876,Alteration: NF1:.| Alteration: NF1:del,Cancer Genome Interpreter,
...,...,...,...,...,...,...,...
299,CISPLATIN,MPHOSPH8,True,0.003043,,Drug Target Commons,
300,CISPLATIN,MDM2,True,0.013811,Alteration: MDM2:amp,Cancer Genome Interpreter,
301,CISPLATIN,MLLT3,True,0.179547,,PharmGKB - The Pharmacogenomics Knowledgebase,
302,CISPLATIN,MAPT,True,0.000814,,Drug Target Commons,


In [3]:
drug_apps = dgipy.get_drug_applications(drugs)
drug_apps

,drug,application,description
0,IMATINIB,ANDA078340,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...
1,IMATINIB,ANDA079179,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
2,IMATINIB,ANDA204285,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
3,IMATINIB,ANDA204644,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...
4,IMATINIB,ANDA206547,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
5,IMATINIB,ANDA207495,IMATINIB MESYLATE: EQ 100MG BASE Discontinued ...
6,IMATINIB,ANDA207586,none
7,IMATINIB,ANDA208302,IMATINIB MESYLATE: EQ 400MG BASE Prescription ...
8,IMATINIB,ANDA208429,none
9,IMATINIB,ANDA210658,IMATINIB MESYLATE: EQ 100MG BASE Prescription ...


## Load FDA Data, Filter Search Space
Load previously downloaded FDA data set (JSON format). Define search space using the previously obtained ANDA/NDA applications via DGIpy. Filter the FDA data set using this search space definition. NOTE: I think it will be important to maintain the *brand_name* and *application_number* sections throughout any downstream data manipulation to maintain the ability to somehow link it back to paired DGI data.  
  
I also think this method would be used to obtain search chunks for any question answering or generative tasks that may be desirable (ala RAG technique that James presented).

In [4]:
# from datasets import Dataset
import pandas as pd
drugs_at_fda = pd.read_excel('../data/openfda-2.xlsx').reset_index(drop=True).drop('Unnamed: 0', axis=1)

relevant_labels = list(set(drug_apps['application']))
search_set = drugs_at_fda[drugs_at_fda['application_number'].isin(relevant_labels)].reset_index(drop=True)
search_set[0:5]

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Imatinib,ANDA206547,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None . None ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
1,Cisplatin,ANDA207323,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNINGS Cisplatin should be administered unde...,NaN,NaN
2,Imatinib Mesylate,ANDA207495,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. (4),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
3,Imatinib Mesylate,ANDA204644,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
4,Imatinib Mesylate,ANDA207586,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN


## LLM Methods
Previous workflow test used a rudimentary implementation of the huggingface ecosystem. For this instance, test some of the gpt4all local models to see if this is a feasible application. The model we will use first is the general purpose *mistral-7b-instruct-v0*

In [21]:
import llm
model = llm.get_model('mistral-7b-instruct-v0')
conversation = model.conversation()

### Indications
Separate into chunks, can you answer questions about the chunks? Also can you extract information about them?

In [9]:
# Clinical Pharmacology section for BRAFTOVI
test = drugs_at_fda[drugs_at_fda['brand_name']=="BRAFTOVI"].reset_index(drop=True)['indications_and_usage'][0]
print(len(test))
test


1416


'1 INDICATIONS AND USAGE BRAFTOVI is a kinase inhibitor indicated: • in combination with binimetinib, for the treatment of patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test. ( 1.1 , 2.1 ) • in combination with cetuximab, for the treatment of adult patients with metastatic colorectal cancer (CRC) with a BRAF V600E mutation, as detected by an FDA-approved test, after prior therapy. ( 1.2 , 2.1 ) Limitations of Use BRAFTOVI is not indicated for treatment of patients with wild-type BRAF melanoma or wild-type BRAF CRC. ( 1.3 , 5.2 ) 1.1 BRAF V600E or V600K Mutation-Positive Unresectable or Metastatic Melanoma BRAFTOVI ® is indicated, in combination with binimetinib, for the treatment of patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test [see Dosage and Administration (2.1) ] . 1.2 BRAF V600E Mutation-Positive Metastatic Colorectal Cancer (CRC) BRAF

In [10]:
response = conversation.prompt(f'Summarize Text Chunk #1: {test}')
print(response)

 The text describes the indications and limitations of use for Braftovi, a kinase inhibitor used to treat unresectable or metastatic melanoma with a BRAF V600E or V600K mutation as detected by an FDA-approved test. It is also indicated in combination with cetuximab for the treatment of adult patients with metastatic colorectal cancer (CRC) with a BRAF V600E mutation, after prior therapy. However, it is not indicated for treatment of patients with wild-type BRAF melanoma or CRC.


In [11]:
response2 = conversation.prompt('from the previous text chunk, identify the subject therapeutic, its indications for use, and any additional interactions.')
response2

<Response prompt='from the previous text chunk, identify the subject therapeutic, its indications for use, and any additional interactions.' text=' The subject therapeutic in this text chunk is Braftovi, a kinase inhibitor. Its indications for use are:
- In combination with binimetinib, for the treatment of patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test.
- In combination with cetuximab, for the treatment of adult patients with metastatic colorectal cancer (CRC) with a BRAF V600E mutation, after prior therapy.
It is important to note that Braftovi is not indicated for treatment of patients with wild-type BRAF melanoma or CRC. Additionally, there are no additional interactions mentioned in this text chunk.'>

In [12]:
response3 = conversation.prompt('Format your previous answer as JSON. Be sure to label the subject therapeutic, indications, and any additional interactions.')
print(response3)

 {
"Subject Therapeutic": "Braftovi",
"Indications for Use": [
{
"Melanoma with BRAF V600E or V600K mutation": true
},
{
"Metastatic colorectal cancer (CRC) with BRAF V600E mutation after prior therapy": true
}
],
"Additional Interactions": []
}


In [13]:
response4 = conversation.prompt('from text chunk #1, identify the subject therapeutic, its indications for use, and any additional interactions. Format your response in JSON.')
print(response4)

 {
"Subject Therapeutic": "Braftovi",
"Indications for Use": [
{
"Melanoma with BRAF V600E or V600K mutation": true
},
{
"Metastatic colorectal cancer (CRC) with BRAF V600E mutation after prior therapy": true
}
],
"Additional Interactions": []
}


### Indications, Again

In [15]:
# Clinical Pharmacology section for BRAFTOVI
test = drugs_at_fda[drugs_at_fda['brand_name']=="BRAFTOVI"].reset_index(drop=True)['indications_and_usage'][0]
print(len(test))
test


1416


'1 INDICATIONS AND USAGE BRAFTOVI is a kinase inhibitor indicated: • in combination with binimetinib, for the treatment of patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test. ( 1.1 , 2.1 ) • in combination with cetuximab, for the treatment of adult patients with metastatic colorectal cancer (CRC) with a BRAF V600E mutation, as detected by an FDA-approved test, after prior therapy. ( 1.2 , 2.1 ) Limitations of Use BRAFTOVI is not indicated for treatment of patients with wild-type BRAF melanoma or wild-type BRAF CRC. ( 1.3 , 5.2 ) 1.1 BRAF V600E or V600K Mutation-Positive Unresectable or Metastatic Melanoma BRAFTOVI ® is indicated, in combination with binimetinib, for the treatment of patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test [see Dosage and Administration (2.1) ] . 1.2 BRAF V600E Mutation-Positive Metastatic Colorectal Cancer (CRC) BRAF

In [17]:
# New conversation
conversation = model.conversation()

In [18]:
response = conversation.prompt(f'Summarize Text Chunk #1: {test}')
print(response)

 The text chunk provides information about the indications and limitations of use for Braftovi, a kinase inhibitor used to treat patients with unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test. It is also indicated in combination with cetuximab for the treatment of adult patients with metastatic colorectal cancer (CRC) with a BRAF V600E mutation, after prior therapy. However, it is not indicated for treatment of patients with wild-type BRAF melanoma or wild-type BRAF CRC.


In [14]:
print(f'this is my intended JSON format: {{"subject_therapeutic": <value>, "indications_for_use": <value>, "additional_interactions": <value>}}')

this is an intended JSON format: {"subject_therapeutic": <value>, "indications_for_use": <value>, "additional_interactions": <value>}


In [19]:
response2 = conversation.prompt(f'This is my intended JSON format: {{"subject_therapeutic": <value>, "indications_for_use": <value>, "additional_interactions": <value>}}. From the previous text chunk, identify the subject therapeutic, its indications for use, and any additional interactions. Format your answer using my intended JSON format.')
response2

<Response prompt='This is my intended JSON format: {"subject_therapeutic": <value>, "indications_for_use": <value>, "additional_interactions": <value>}. From the previous text chunk, identify the subject therapeutic, its indications for use, and any additional interactions. Format your answer using my intended JSON format.' text=' {"subject_therapeutic": "Braftovi", "indications_for_use": ["unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test", "metastatic colorectal cancer (CRC) with a BRAF V600E mutation, after prior therapy"], "additional_interactions": []}'>

In [22]:
response2.text()

' {"subject_therapeutic": "Braftovi", "indications_for_use": ["unresectable or metastatic melanoma with a BRAF V600E or V600K mutation, as detected by an FDA-approved test", "metastatic colorectal cancer (CRC) with a BRAF V600E mutation, after prior therapy"], "additional_interactions": []}'

## Batch Indications
Single test for indications seems potentially good, let's try batching all our therapeutics and find out where the roadblocks happen. I suspect figuring out the context window and how to properly chunk longer sections will be the biggest issue.

### Attempt 1

In [6]:
indication_search = search_set[search_set['indications_and_usage'].isnull()==False].reset_index(drop=True)
indication_search[0:5]

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Imatinib,ANDA206547,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None . None ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
1,Cisplatin,ANDA207323,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNINGS Cisplatin should be administered unde...,NaN,NaN
2,Imatinib Mesylate,ANDA207495,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. (4),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
3,Imatinib Mesylate,ANDA204644,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
4,Imatinib Mesylate,ANDA207586,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN


In [7]:
import llm
model = llm.get_model('mistral-7b-instruct-v0')

def ask_mistral(text): # Something weird about being inside a function????????
    conversation = model.conversation()
    response = conversation.prompt(f'Summarize Text Chunk #1: {text}')
    response2 = conversation.prompt(f'This is my intended JSON format: {{"subject_therapeutic": <value>, "indications_for_use": <value>, "additional_interactions": <value>}}. From Text Chunk #1, identify the subject therapeutic, its indications for use, and any additional interactions. Format your answer using my intended JSON format.')

    return(response2.text())

In [8]:
indication_search['json'] = indication_search['indications_and_usage'].apply(ask_mistral)


In [9]:
indication_search[['brand_name','indications_and_usage','json']]

,brand_name,indications_and_usage,json
0,Imatinib,1 INDICATIONS AND USAGE Imatinib mesylate is a...,"{\n ""subject_therapeutic"": ""Antidepressants""..."
1,Cisplatin,INDICATIONS Cisplatin Injection is indicated a...,"{""subject_therapeutic"": ""Statins"", ""indicatio..."
2,Imatinib Mesylate,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,"{""subject_therapeutic"": ""Vitamin D"", ""indicat..."
3,Imatinib Mesylate,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,"{""subject_therapeutic"": ""Acetaminophen"", ""ind..."
4,Imatinib Mesylate,1 INDICATIONS AND USAGE Imatinib mesylate is a...,"{""subject_therapeutic"": ""Statins"", ""indicatio..."
5,CISplatin,INDICATIONS Cisplatin Injection is indicated a...,"{""subject_therapeutic"": ""Statins"", ""indicatio..."
6,VAZALORE,Uses Uses For temporary relief of minor aches ...,"{""subject_therapeutic"": ""Statins"", ""indicatio..."
7,Imatinib Mesylate,1 INDICATIONS AND USAGE Imatinib mesylate is a...,"{""subject_therapeutic"": ""Antidepressants"", ""i..."
8,BRAFTOVI,1 INDICATIONS AND USAGE BRAFTOVI is a kinase i...,"{\n ""subject_therapeutic"": ""Antidepressants""..."
9,imatinib mesylate,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,"{""subject_therapeutic"": ""Antidepressants"", ""i..."


In [10]:
import ast
ast.literal_eval(indication_search['json'][0])

{'subject_therapeutic': 'Antidepressants',
 'indications_for_use': 'Treatment of depression, anxiety disorders, and other psychiatric conditions',
 'additional_interactions': 'May interact with certain medications such as blood thinners, sedatives, and stimulants'}

In [51]:
indication_search['indications_and_usage'][0]

'1 INDICATIONS AND USAGE Imatinib mesylate is a kinase inhibitor indicated for the treatment of: Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase (1.1) Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis (BC), accelerated phase (AP), or in chronic phase (CP) after failure of interferon-alpha therapy (1.2) Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) (1.3) Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy (1.4) Adult patients with myelodysplastic/myeloproliferative diseases (MDS/MPD) associated with platelet-derived growth factor receptor (PDGFR) gene re-arrangements (1.5) Adult patients with aggressive systemic mastocytosis (ASM) without the D816V c-Kit mutation or with c-Kit mutational status u

### Attempt #2

In [5]:
indication_search = search_set[search_set['indications_and_usage'].isnull()==False].reset_index(drop=True)
indication_search[0:5]

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics
0,Imatinib,ANDA206547,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None . None ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
1,Cisplatin,ANDA207323,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNINGS Cisplatin should be administered unde...,NaN,NaN
2,Imatinib Mesylate,ANDA207495,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. (4),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
3,Imatinib Mesylate,ANDA204644,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN
4,Imatinib Mesylate,ANDA207586,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN


In [46]:
# # Test block
# model = 'reset'
# model = llm.get_model('mistral-7b-instruct-v0')
# text = indication_search["indications_and_usage"][0]
# response2 = model.prompt(f'Read the following text and identify the subject therapeutic, its indications for use, and any additional interactions. Keep your indications brief and format your answer using this JSON format: {{"subject_therapeutic": <value>, "indications_for_use": [<value>], "additional_interactions": [<value>]}}. \n Text: {text} ')

# # response = conversation.prompt(f'Remember this text as Chunk 1: {text}')
# # response2 = conversation.prompt(f'Read the following text and identify the subject therapeutic, its indications for use, and any additional interactions. Keep your indications brief and format your answer using this JSON format: {{"subject_therapeutic": <value>, "indications_for_use": [<value>], "additional_interactions": [<value>]}}. \n Text: {text} ')
# print(response2.text())


 {"subject_therapeutic": "Imatinib mesylate", "indications_for_use": ["Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase (1.1)",
"Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis (BC), accelerated phase (AP), or in chronic phase (CP) after failure of interferon-alpha therapy (1.2)",
"Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ALL) (1.3)",
"Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy (1.4)",



In [6]:
import llm
model = llm.get_model('mistral-7b-instruct-v0')
from tqdm import tqdm


index_pos = 0
indication_search['json'] = 'empty'

for indication in tqdm(indication_search['indications_and_usage']):
    model = 'reset'
    model = llm.get_model('mistral-7b-instruct-v0')

    response = model.prompt(f'Read the following text and identify the subject therapeutic, its indications for use, and any additional interactions. Format your answer using this JSON format: {{"subject_therapeutic": <value>, "indications_for_use": [<value>]}}. \n Text: {indication}. Be sure to close your JSON tags. ')

    print(response.text())
    indication_search['json'][index_pos] = response.text()
    index_pos += 1





  3%|▎         | 1/38 [04:51<2:59:45, 291.51s/it]

 {"subject_therapeutic": "Imatinib mesylate", "indications_for_use": ["Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+CML) in chronic phase (1.1)",
"Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis (BC), accelerated phase (AP), or in chronic phase (CP) after failure of interferon-alpha therapy (1.2)",
"Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ALL) (1.3)",
"Pediatric patients with newly diagnosed Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL) in combination with chemotherapy (1.4)",



  5%|▌         | 2/38 [06:13<1:40:53, 168.15s/it]

 {"subject_therapeutic": "Cisplatin Injection", "indications_for_use": ["Metastatic Testicular Tumors", "Established combination therapy with other approved chemotherapeutic agents in patients who have already received appropriate surgical and/or radiotherapeutic procedures.", 
"Metastatic Ovarian Tumors", "Established combination therapy with other approved chemotherapeutic agents in patients who have already received appropriate surgical and/or radiotherapeutic procedures. An established combination consists of cisplatin and cyclophosphamide.", 
"Advanced Bladder Cancer", "Single agent for patients with transitional cell bladder cancer which is no longer amenable to local treatments, such as surgery and/or radiotherapy."]


  8%|▊         | 3/38 [10:50<2:07:12, 218.08s/it]

...

 {"subject_therapeutic": "Cisplatin injection", "indications_for_use": ["Advanced testicular cancer", "Advanced ovarian cancer", "Advanced bladder cancer"]}


 97%|█████████▋| 37/38 [2:13:21<02:43, 163.68s/it]

 {"subject_therapeutic": "Cisplatin Injection", "indications_for_use": ["Metastatic Testicular Tumors", "Established combination therapy with other approved chemotherapeutic agents in patients who have already received appropriate surgical and/or radio therapeutic procedures.", 
"Metastatic Ovarian Tumors", "Established combination therapy with other approved chemotherapeutic agents in patients who have already received appropriate surgical and/or radiotherapeutic procedures. An established combination consists of cisplatin and cyclophosphamide.", 
"Advanced Bladder Cancer", "Single agent for patients with transitional cell bladder cancer which is no longer amenable to local treatments, such as surgery and/or radiotherapy."]


100%|██████████| 38/38 [2:18:02<00:00, 217.95s/it]

 {"subject_therapeutic": "Imatinib mesylate", "indications_for_use": ["Newly diagnosed adult and pediatric patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in chronic phase", "Patients with Philadelphia chromosome positive chronic myeloid leukemia (Ph+ CML) in blast crisis, accelerated phase, or in chronic phase after failure of interferon-alpha therapy", "Adult patients with relapsed or refractory Philadelphia chromosome positive acute lymphoblastic leukemia (Ph+ ALL)", "Adult patients with myelodysplastic/myeloproliferative diseases associated with PDGFR gene rearrangements as determined with an FDA-approved test", "Adult patients with aggressive systemic mastocytosis without the D816V c-Kit mutation as


In [8]:
indication_search

,brand_name,application_number,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,warnings,precautions,pharmacokinetics,purpose,clinical_pharmacology,active_ingredient,stop_use,boxed_warning,pharmacodynamics,pharmacogenomics,json
0,Imatinib,ANDA206547,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None . None ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""subject_therapeutic"": ""Imatinib mesylate"", ..."
1,Cisplatin,ANDA207323,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNINGS Cisplatin should be administered unde...,NaN,NaN,"{""subject_therapeutic"": ""Cisplatin Injection""..."
2,Imatinib Mesylate,ANDA207495,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. (4),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""subject_therapeutic"": ""Imatinib mesylate ta..."
3,Imatinib Mesylate,ANDA204644,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate tabl...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS Edema and severe fl...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""subject_therapeutic"": ""Imatinib mesylate ta..."
4,Imatinib Mesylate,ANDA207586,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,NaN,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""subject_therapeutic"": ""Imatinib mesylate"", ..."
5,CISplatin,ANDA075036,ADVERSE REACTIONS Nephrotoxicity Dose-related ...,INDICATIONS Cisplatin Injection is indicated a...,CONTRAINDICATIONS Cisplatin is contraindicated...,NaN,WARNINGS Cisplatin produces cumulative nephrot...,PRECAUTIONS Peripheral blood counts should be ...,NaN,NaN,CLINICAL PHARMACOLOGY Plasma concentrations of...,NaN,NaN,WARNING Cisplatin Injection should be administ...,NaN,NaN,"{""subject_therapeutic"": ""Cisplatin Injection""..."
6,VAZALORE,NDA203697,NaN,Uses Uses For temporary relief of minor aches ...,NaN,NaN,WARNINGS Warnings Reye’s Syndrome: Children an...,NaN,NaN,Active Ingredient (in Each capsules) Purposes:...,NaN,ACTIVE iNGREDIENT Active Ingredient (in Each c...,Do not use I f you have ever had an allergic r...,NaN,NaN,NaN,"{""subject_therapeutic"": ""aspirin"", ""indicatio..."
7,Imatinib Mesylate,ANDA079179,6 ADVERSE REACTIONS The following serious adve...,1 INDICATIONS AND USAGE Imatinib mesylate is a...,4 CONTRAINDICATIONS None. None ( 4 ).,5 WARNINGS AND PRECAUTIONS • Edema and severe ...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,NaN,NaN,"{""subject_therapeutic"": ""Imatinib mesylate"", ..."
8,BRAFTOVI,NDA210496,6 ADVERSE REACTIONS The following adverse reac...,1 INDICATIONS AND USAGE BRAFTOVI is a kinase i...,4 CONTRAINDICATIONS None. None. ( 4 ),5 WARNINGS AND PRECAUTIONS • New Primary Malig...,NaN,NaN,12.3 Pharmacokinetics The pharmacokinetics of ...,NaN,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,NaN,NaN,NaN,12.2 Pharmacodynamics Cardiac Electrophysiolog...,NaN,"{""subject_therapeutic"": ""BRAFTOVI"", ""indicati..."
9,imatinib mesylate,ANDA212135,6 ADVERSE REACTIONS T

### Scratch